In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import spacy
# nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
df = pd.read_csv('../data/processed_text.csv')
samples = df[['Grade','Text']]
train_set_size = .8
idx=np.arange(len(samples))
np.random.shuffle(idx)
split_point = int(len(idx)*train_set_size)
train_samples = samples.iloc[idx[:split_point]]
test_samples = samples.iloc[idx[split_point:]]

In [3]:
train_samples

,Grade,Text
65,6.0,In the beginning my people were primitive an...
67,6.0,More than any other state Hawaii is world fam...
297,3.0,"One night when the air was warm, my puppies we..."
138,6.0,I cant remember the first time I climbed that ...
203,8.0,Great people of North Dakota I Senator ...
...,...,...
139,6.0,The explosions and gunshots were definitely ge...
276,2.0,I think that Thomas Alva Edison is the most im...
112,4.0,I will always love my grandparents beach house...
318,10.0,The violin is arguably the most cherished and ...


!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [13]:
tokens[1]

My cat is fluffy. His name is Buzz. He is my favorite cat. Buzz is my favorite pet of all. I hug him all the time. He is soft  and his cheeks are fat like my brothers. I feel happy with my cat.

In [11]:
nlp = spacy.load('en_core_web_sm')
# tokens = [nlp(text) for text in df.Text]
for token in tokens:
    print(token.lemma_)


AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'lemma_'

In [ ]:

feature = 'Tokens'
X = df[feature]
y= df.Grade
samples = df[['Grade',feature]]
train_set_size = .8
idx=np.arange(len(X))
np.random.shuffle(idx)
split_point = int(len(idx)*train_set_size)
train_samples = samples.iloc[idx[:split_point]]
test_samples = samples.iloc[idx[split_point:]]


vectorizer = TextVectorization(output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples.to_numpy()).batch(64)
vectorizer.adapt(text_ds)
# X_train=vectorizer(np.array([[s] for s in train_samples])).numpy()
# y_train = y[idx[:split_point]]

# X_test=vectorizer(np.array([[s] for s in test_samples])).numpy()

# y_test = y[idx[split_point:]]
# X_test

In [ ]:
word2vec = Word2Vec(df.Tokens, min_count=1)
vocabulary = word2vec.wv.vocab

In [ ]:
word2vec.wv.most_similar('school')

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

Thanks to https://keras.io/examples/nlp/pretrained_word_embeddings/ for the below

In [ ]:
words = vocabulary.keys()
word_index = dict(zip(words, range(len(words))))

In [ ]:
len(word_index.keys())

In [ ]:
word_index['chobot']

In [ ]:
num_tokens = len(words)+2
embedding_dim = len(glove_vectors['anything'])
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    
    try: 
        embedding_vector = glove_vectors[word]
        embedding_matrix[i] = embedding_vector
        hits += 1
    except:
        misses +=1
print('Converted %d words (%d misses)' % (hits, misses))

In [ ]:
embedding_matrix.shape

In [ ]:
from tensorflow.keras.layers import Embedding
embedding_layer=Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False)

In [ ]:
from tensorflow.keras import layers
int_sequences_input = keras.Input(shape=(None,),dtype="int64")
embedded_sequences=embedding_layer(int_sequences_input)
x=layers.LSTM(10,activation='relu',recurrent_activation='relu')(embedded_sequences)
preds=layers.Dense(1, activation='relu')(x)
model = keras.Model(int_sequences_input, preds)

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae'])
model.summary()



In [ ]:
X_t = np.asarray(X_train).astype('float32')

In [ ]:
X_t = word2vec.transform(np.array([[s] for s in df.Tokens])).numpy()

In [ ]:
train_samples

In [ ]:
model.fit(X_train, y_train,
         batch_size=1,
         epochs=10,
         )

In [ ]:
X_train